In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [12]:
ds=pd.read_csv('mushrooms naive bayes.csv')
ds.shape
ds.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [13]:
le=LabelEncoder()
df=ds.apply(le.fit_transform)
print(df.shape)
df.head()

(8124, 23)


,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


In [14]:
split=int(df.shape[0]*0.90)
split
train=df[:split]
test=df[split:]
test.shape
print(type(test))
print(type(train))
test=test.reset_index(drop=True)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [15]:
print (train.head())
print('--------')
print(test.head())

   type  cap_shape  cap_surface  cap_color  bruises  odor  gill_attachment  \
0     1          5            2          4        1     6                1   
1     0          5            2          9        1     0                1   
2     0          0            2          8        1     3                1   
3     1          5            3          8        1     6                1   
4     0          5            2          3        0     5                1   

   gill_spacing  gill_size  gill_color   ...     stalk_surface_below_ring  \
0             0          1           4   ...                            2   
1             0          0           4   ...                            2   
2             0          0           5   ...                            2   
3             0          1           5   ...                            2   
4             1          0           4   ...                            2   

   stalk_color_above_ring  stalk_color_below_ring  veil_type  veil_c

In [21]:
def prior_prob(data,lc,lv):
    nrow=data[data[lc]==lv]
    classprob=nrow.shape[0]/data.shape[0]
   # print(type(nrow))
    return classprob
print(prior_prob(df,'type',1))
def cond_prob(data,fc,fv,lc,lv):
    crow=data[(data[fc]==fv)&(data[lc]==lv)]
    nrow=data[data[lc]==lv]
    columncondprob=crow.shape[0]/nrow.shape[0]
    print(data[0][0])
    return columncondprob
print(cond_prob(df,'odor',8,'type',1))
#print(data.loc[0][0])

0.48202855736090594


KeyError: 0

In [7]:
temp=df.values[:,0]
print(temp)
x=np.unique(temp,return_counts=True)
print(x)
print((x[1][0])/x[1].sum())
print(test.loc[213])

[1 0 0 ... 0 1 0]
(array([0, 1], dtype=int32), array([4208, 3916]))
0.517971442639094
type                        0
cap_shape                   0
cap_surface                 2
cap_color                   8
bruises                     0
odor                        5
gill_attachment             1
gill_spacing                1
gill_size                   0
gill_color                  2
stalk_shape                 0
stalk_root                  0
stalk_surface_above_ring    1
stalk_surface_below_ring    1
stalk_color_above_ring      7
stalk_color_below_ring      7
veil_type                   0
veil_color                  2
ring_number                 2
ring_type                   4
spore_print_color           7
population                  3
habitat                     1
Name: 213, dtype: int32


In [8]:
CLASSES=np.unique(df['type'])
pred=[]
cx=1
for i in range(test.shape[0]):
    
    if(cx%100==0):
        print(cx)
        cx+=1
    val=test.loc[i][1:]
    
    feat=dict(val)
    posteriorprobs = []
    for c in CLASSES:
        like=1
        for f in feat:
            cp=cond_prob(train,f,feat[f],'type',c)
            like*=cp
        priorclassprob=prior_prob(train,'type',c)
        postprob=priorclassprob*like
        posteriorprobs.append(postprob)
        
    posteriorprobs=np.array(posteriorprobs)
    pred_label=posteriorprobs.argmax()
    
    pred.append(pred_label)
    
    
    

In [89]:
print(len(pred))
truelab=np.array(test['type'])

predlab=np.array(pred)
accuracy=np.sum(np.array(truelab==predlab))/truelab.shape[0]
print(accuracy)

813
0.97539975399754


In [90]:
temp = test.loc[0][1:]
print (temp.shape)
fa = dict(temp)
print (fa)
print(type(test))


(22,)
{'cap_shape': 5, 'cap_surface': 0, 'cap_color': 8, 'bruises': 0, 'odor': 5, 'gill_attachment': 1, 'gill_spacing': 1, 'gill_size': 0, 'gill_color': 2, 'stalk_shape': 0, 'stalk_root': 0, 'stalk_surface_above_ring': 2, 'stalk_surface_below_ring': 2, 'stalk_color_above_ring': 7, 'stalk_color_below_ring': 7, 'veil_type': 0, 'veil_color': 2, 'ring_number': 2, 'ring_type': 4, 'spore_print_color': 7, 'population': 3, 'habitat': 1}
<class 'pandas.core.frame.DataFrame'>


In [91]:
posteriorprobs1 = []

In [174]:
'''Homework Assignment'''
l2=[]
def get_testpoint(tp):
    CLASSES=np.unique(df['type'])
    pred=[]
    val=test.loc[tp][1:]

    feat=dict(val)
    posteriorprobs1 = []
    for c in CLASSES:
        like=1
        for f in feat:
            cp=cond_prob(train,f,feat[f],'type',c)
            like*=cp
        priorclassprob1=prior_prob(train,'type',c)
        postprob1=priorclassprob*like
        
        posteriorprobs1.append(postprob1)
    posteriorprobs1=np.array(posteriorprobs1)
    totalprob=posteriorprobs1.sum()
    
    pred_label=posteriorprobs1.argmax()
    confidence=posteriorprobs1[pred_label]/totalprob

    pred.append(pred_label)
    truelab=np.array(test['type'])

    predlab=np.array(pred)
    accuracy=np.sum(np.array(truelab==predlab))/truelab.shape[0]
    return confidence,predlab

    
       
        
    
    
def get_prediction_and_confidence(test_point):
    '''Code Goes here'''
    get_testpoint(test_point)
    
    #return pred, conf

for j in range(test.shape[0]):
    l2.append(get_testpoint(j))
    #print(l2)
p=np.array(l2)
print(p.shape)
print(p[:,0])
x2=np.unique(p[:,0],return_counts=True)
print("..........")
print(x2)




    

(813, 2)
[0.99953403 1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         0.99992073
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         0.99995947
 1.         1.         0.90058559 0.99938148 1.         1.
 0.99819814 1.         1.         1.         1.         1.
 0.52216085 0.9998183  0.99984963 1.         1.         1.
 1.         1.         1.         1.         1.         0.99408187
 1.         1.         0.99148159 1.         1.         1.
 0.99901252 0.99617004 1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         0.86141968 0.82648348 1.         1.         1.
 1.         1.         1.         0.55250501 1.         0.99356992
 1.         1.         1.         0.99797206 1.         1.
 1.         1. 